In [1]:
!pip install crossrefapi

  Preparing metadata (setup.py) ... done
  Created wheel for crossrefapi: filename=crossrefapi-1.5.0-py3-none-any.whl size=15693 sha256=1d2adfb90010859a2a8ccee296609500af91ca7d6b2d2a80a542d76cec2221c6
  Stored in directory: /private/tmp/pip-ephem-wheel-cache-qd0g03rw/wheels/d1/2b/00/95f121ec56d164aac41ddf469fe421d347d4501ac773ab28b1
Successfully built crossrefapi


In [9]:
!pip3 install crossref-commons

  Preparing metadata (setup.py) ... done
  Created wheel for ratelimit: filename=ratelimit-2.2.1-py3-none-any.whl size=5895 sha256=4df7ba5270bd8fc9edb3a56bde90eb2c9d986d9b931118407815d66072f6c13d
  Stored in directory: /private/tmp/pip-ephem-wheel-cache-pyw9yd36/wheels/14/1e/97/126009a0884bdf7e26436cace73d9a4f4596dada4fdc4950ce
Successfully built ratelimit


In [1]:
!pip install orcid-python

In [52]:
!pip install python-benedict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 20.8 MB/s eta 0:00:0000:010:01
  Created wheel for mailchecker: filename=mailchecker-5.0.7-py3-none-any.whl size=322214 sha256=243d67050ed64c3c0bedce9df11592d399a1f3f838b42d85650dc1e880268496
  Stored in directory: /private/tmp/pip-ephem-wheel-cache-t0agg3wf/wheels/9a/62/46/70a2b38635e9cf1ec55b2af389ff0c7baac750803810fb2c83
Successfully built mailchecker
  Attempting uninstall: python-slugify
    Found existing installation: python-slugify 5.0.2
    Uninstalling python-slugify-5.0.2:


In [54]:
from crossref.restful import Works
import pandas as pd
import numpy as np
import json
import requests
import urllib.request
import matplotlib.pyplot as plt
import requests
import seaborn as snb
import datetime
import itertools
from collections import Counter
import networkx as nx
from pyvis.network import Network
import benedict
import orcid

In [10]:
import crossref_commons.retrieval

In [123]:
import sys
sys.executable

'/Users/jennifer/opt/anaconda3/bin/python'

In [62]:
works = Works()

In [41]:
ewater = works.doi('10.5194/gmd-15-5371-2022')
ewater

{'indexed': {'date-parts': [[2022, 12, 13]],
  'date-time': '2022-12-13T00:29:40Z',
  'timestamp': 1670891380257},
 'reference-count': 48,
 'publisher': 'Copernicus GmbH',
 'issue': '13',
 'license': [{'start': {'date-parts': [[2022, 7, 14]],
    'date-time': '2022-07-14T00:00:00Z',
    'timestamp': 1657756800000},
   'content-version': 'vor',
   'delay-in-days': 0,
   'URL': 'https://creativecommons.org/licenses/by/4.0/'}],
 'funder': [{'DOI': '10.13039/100013407',
   'name': 'Netherlands eScience Center',
   'doi-asserted-by': 'publisher',
   'award': ['027.017.F01']}],
 'content-domain': {'domain': [], 'crossmark-restriction': False},
 'short-container-title': ['Geosci. Model Dev.'],
 'abstract': '<jats:p>Abstract. Hutton et\xa0al. (2016) argued that computational hydrology can only be a proper science if the hydrological community makes sure that hydrological model studies are executed and presented in a reproducible manner. Hut, Drost and van de Giesen replied that to achieve this

In [66]:
works.doi('10.5194/wes-4-193-2019')['author']

[{'ORCID': 'http://orcid.org/0000-0002-5025-7862',
  'authenticated-orcid': False,
  'given': 'Peter C.',
  'family': 'Kalverla',
  'sequence': 'first',
  'affiliation': []},
 {'ORCID': 'http://orcid.org/0000-0002-8882-5693',
  'authenticated-orcid': False,
  'given': 'James B.',
  'family': 'Duncan Jr.',
  'sequence': 'additional',
  'affiliation': []},
 {'ORCID': 'http://orcid.org/0000-0002-5922-8179',
  'authenticated-orcid': False,
  'given': 'Gert-Jan',
  'family': 'Steeneveld',
  'sequence': 'additional',
  'affiliation': []},
 {'ORCID': 'http://orcid.org/0000-0003-0995-2481',
  'authenticated-orcid': False,
  'given': 'Albert A. M.',
  'family': 'Holtslag',
  'sequence': 'additional',
  'affiliation': []}]

In [56]:
authors = ewater['author']
orcid_list = []
author_names = []
for i in authors:
    if i['ORCID']:
        orcid_list.append(i['ORCID'])
    else:
        
    author_names.append(i['given']+' '+i['family'])
orcid_list

['Rolf Hut',
 'Niels Drost',
 'Nick van de Giesen',
 'Ben van Werkhoven',
 'Banafsheh Abdollahi',
 'Jerom Aerts',
 'Thomas Albers',
 'Fakhereh Alidoost',
 'Bouwe Andela',
 'Jaro Camphuijsen',
 'Yifat Dzigan',
 'Ronald van Haren',
 'Eric Hutton',
 'Peter Kalverla',
 'Maarten van Meersbergen',
 'Gijs van den Oord',
 'Inti Pelupessy',
 'Stef Smeets',
 'Stefan Verhoeven',
 'Martine de Vos',
 'Berend Weel']

AttributeError: module 'orcid' has no attribute 'search'

In [124]:
pwd

'/Users/jennifer/RCSNA'

In [17]:
# Query ORCID by a fullname
headers = {
    'Accept': 'application/vnd.orcid+json',
}
def name_to_orcid_id(name):
    orcid_id = None
    
    params = (
        ('q',name),
    )
    response = requests.get('https://pub.orcid.org/v3.0/search/', headers=headers, params=params)
    temp = response.json()
    if len(temp):
        orcid_id = temp['result'][0]['orcid-identifier']['path']
    return orcid_id

In [19]:
name_to_orcid_id('Gijs van den Oord')

'0000-0001-8367-1333'

In [20]:
name_to_orcid_id('Peter Kalverla')

'0000-0002-5025-7862'

In [95]:
#
def get_authors(doi):
    works = Works()
    metadata = works.doi(doi)
    if (metadata) and ('author' in metadata.keys()):
        authors = metadata['author']
    
        orcid_list = []
        author_names = []

        for i in authors:
            if 'ORCID' in i.keys():
                orcid_id_split = i['ORCID'].split("http://orcid.org/")[1]
                orcid_list.append(orcid_id_split)
            else:
                orcid_id_by_name = name_to_orcid_id(i['given']+' '+i['family'])
                orcid_list.append(orcid_id_by_name)
            author_names.append(i['given']+' '+i['family'])
        return orcid_list, author_names
    else:
        return [],[]

In [109]:
orcid, name = get_authors('10.5194/gmd-15-5371-2022')

In [110]:
orcid

['0000-0002-5697-5697',
 '0000-0001-9795-7981',
 '0000-0002-7200-3353',
 '0000-0002-7508-3272',
 '0000-0002-8969-3821',
 '0000-0003-0157-4818',
 '0000-0003-1182-8543',
 '0000-0001-8407-6472',
 '0000-0001-9005-8940',
 '0000-0002-8928-7831',
 '0000-0002-0935-0088',
 '0000-0002-3160-3547',
 '0000-0002-5864-6459',
 '0000-0002-5025-7862',
 '0000-0002-9585-4142',
 '0000-0001-8367-1333',
 '0000-0002-8024-0412',
 '0000-0002-5413-9038',
 '0000-0002-5821-2060',
 '0000-0001-5301-1713',
 '0000-0002-9693-9332']

In [74]:
name

['Rolf Hut',
 'Niels Drost',
 'Nick van de Giesen',
 'Ben van Werkhoven',
 'Banafsheh Abdollahi',
 'Jerom Aerts',
 'Thomas Albers',
 'Fakhereh Alidoost',
 'Bouwe Andela',
 'Jaro Camphuijsen',
 'Yifat Dzigan',
 'Ronald van Haren',
 'Eric Hutton',
 'Peter Kalverla',
 'Maarten van Meersbergen',
 'Gijs van den Oord',
 'Inti Pelupessy',
 'Stef Smeets',
 'Stefan Verhoeven',
 'Martine de Vos',
 'Berend Weel']

In [44]:
# URL for ORCID API
ORCID_RECORD_API = "https://pub.orcid.org/v3.0/"

# query ORCID for an ORCID record
def query_orcid_for_record(orcid_id):

    response = requests.get(url = requests.utils.requote_uri(ORCID_RECORD_API + orcid_id),
                          headers = {'Accept': 'application/json'})
    response.raise_for_status()
    result=response.json()
    return result

# query author name from ORCID
def from_orcid_to_name(orcid_id):
    orcid_record = query_orcid_for_record(orcid_id)
    name_attr = orcid_record['person']['name']
    name = name_attr['given-names']['value'] + ' ' + name_attr['family-name']['value']

In [56]:
orcid_record = query_orcid_for_record('0000-0002-5025-7862')

In [57]:
orcid_record['person']['name']['given-names']['value']+' '+orcid_record['person']['name']['family-name']['value']

'Peter Kalverla'

In [58]:
orcid_record

{'orcid-identifier': {'uri': 'https://orcid.org/0000-0002-5025-7862',
  'path': '0000-0002-5025-7862',
  'host': 'orcid.org'},
 'preferences': {'locale': 'en'},
 'history': {'creation-method': 'DIRECT',
  'completion-date': None,
  'submission-date': {'value': 1440847595525},
  'last-modified-date': {'value': 1677858209460},
  'claimed': True,
  'source': None,
  'deactivation-date': None,
  'verified-email': True,
  'verified-primary-email': True},
 'person': {'last-modified-date': {'value': 1614867500639},
  'name': {'created-date': {'value': 1460759791637},
   'last-modified-date': {'value': 1460759791637},
   'given-names': {'value': 'Peter'},
   'family-name': {'value': 'Kalverla'},
   'credit-name': None,
   'source': None,
   'visibility': 'public',
   'path': '0000-0002-5025-7862'},
  'other-names': {'last-modified-date': None,
   'other-name': [],
   'path': '/0000-0002-5025-7862/other-names'},
  'biography': None,
  'researcher-urls': {'last-modified-date': None,
   'research

In [104]:
# Extract works from ORCID
def extract_works_section(orcid_record):
    works = orcid_record['activities-summary']['works']['group']
    return works

# Extract title and DOI
def extract_doi(work):
    work_summary = work['work-summary'][0]
    title = work_summary['title']['title']['value']
    dois =  [doi['external-id-value'] for doi in work_summary['external-ids']['external-id'] if doi['external-id-type']=="doi"]
    # if there is a DOI, we can extract the first one
    doi = dois[0] if dois else None
    doi = str(doi)
    return doi, title


works = extract_works_section(orcid_record)
for work in works:
    doi,title = extract_doi(work)
    if doi:
        print(f"{doi}, {title}")
    else:
        print(title)

10.5194/egusphere-egu21-9387, A multi-model ensemble weighting method (ClimWIP) in ESMValTool
10.5194/egusphere-egu21-6051, Preprocessing of hydrological models&#8217; input in eWaterCycle with ESMValTool
10.5194/egusphere-egu21-7797, Towards Open and FAIR Hydrological Modelling with eWaterCycle
10.5194/egusphere-egu21-4805, Bringing ESMValTool to the Jupyter Lab
10.5194/egusphere-egu21-3476, Recent developments on the Earth System Model Evaluation Tool
10.5194/egusphere-egu21-4895, era5cli: the command line interface to ERA5 data
10.5194/wes-5-1097-2020, Clustering wind profile shapes to estimate airborne wind energy production
10.5194/egusphere-egu2020-21619, era5cli: The command line tool to download ERA5 data
10.18174/498797, Characterisation of offshore winds for energy applications
10.5194/wes-4-193-2019, Low-level jets over the North Sea based on ERA5 and observations: together they do better
10.5194/wes-2018-79-supplement, Supplementary material to &quot;Low-level jets over the

In [117]:
for work in works:
    doi,title = extract_doi(work)
    orcid, name = get_authors(doi)
        
#     if meta:
#         continue
#     else:
#         print(meta)

['0000-0002-3986-1268', '0000-0001-5760-4524', '0000-0002-5025-7862', '0000-0002-5413-9038', '0000-0002-8928-7831', '0000-0001-9005-8940']
['0000-0001-8407-6472', '0000-0003-0157-4818', '0000-0001-9005-8940', '0000-0002-8928-7831', '0000-0002-7200-3353', '0000-0001-8367-1333', '0000-0001-9795-7981', '0000-0002-5697-5697', '0000-0002-5025-7862', '0000-0002-8024-0412', '0000-0002-7508-3272', '0000-0002-5413-9038', '0000-0002-5821-2060']
['0000-0001-9795-7981', '0000-0003-0157-4818', '0000-0001-8407-6472', '0000-0001-9005-8940', '0000-0002-8928-7831', '0000-0002-7200-3353', '0000-0002-5697-5697', '0000-0002-5864-6459', '0000-0002-5025-7862', '0000-0001-8367-1333', '0000-0002-8024-0412', '0000-0002-5413-9038', '0000-0002-5821-2060', '0000-0002-7508-3272']
['0000-0002-5025-7862', '0000-0002-5413-9038', '0000-0001-9795-7981', '0000-0001-9005-8940', '0000-0001-8407-6472', '0000-0002-8928-7831']
['0000-0001-9005-8940', '0000-0001-8407-6472', '0000-0001-5760-4524', '0000-0002-8928-7831', '0000-

In [116]:
orcid

['0000-0002-5025-7862',
 '0000-0002-5922-8179',
 '0000-0001-6734-2660',
 '0000-0003-0995-2481']

In [121]:
docs = extract_works_section(orcid_record)
all_orcid = []
all_names = []
for doc in docs:
    dois, titles = extract_doi(doc)
        
    for doi in dois:
            orcid_list, name_list = get_authors(doi)
            all_orcid += orcid_list
            all_names += name_list

In [113]:
def get_coauthors(full_name):
    orcid_id = name_to_orcid_id(full_name)
    orcid_record = query_orcid_for_record(orcid_id)
    docs = extract_works_section(orcid_record)
    all_orcid = []
    all_names = []
        
    for doc in docs:
        dois, titles = extract_doi(doc)
        
        for doi in dois:
            orcid_list, name_list = get_authors(doi)
            all_orcid += orcid_list
            all_names += name_list
        
    df = pd.DataFrame()
    df['orcid'] = all_orcid
    df['name'] = all_names
    new_df = df.drop_duplicates(subset = ['orcid'],keep='first', ignore_index=True)
    
    return new_df
    

In [114]:
my_df = get_coauthors("Peter Kalverla")

In [115]:
my_df

,orcid,name
